# Better Way 45 지역 시간은 time이 아닌 datetime으로 표현하자

* https://github.com/KangJungHwa/Effective-Python/blob/master/item_45.py
* https://github.com/shoark7/Effective-Python/blob/master/files/BetterWay45_UseDatetimeForLocalTime.md


* 협정 세계시(UTC, Coordinated Universal Time)는 시간대에 의존하지 않는 표준 시간 표현. 
* UTC는 유닉스 기원 이후로 지나간 초로 시간을 표현하는 컴퓨터에서 잘 작동.


* 사람이 사용하는 시간은 현재 자신이 있는 위치를 기준으로 함.
* 프로그램에서 시간을 처리해야 한다면 사람이 이해하기 쉽게 UTC와 지역 시간 사이에서 변환이 필요.

* 파이썬은 두 가지 시간대 변환 방법을 제공.
    * 내장 모듈 time을 사용하는 방법은 치명적인 오류가 일어날 가능성이 큼
    * 내장 모듈 datetime을 사용하는 방법은 커뮤니티에서 만든 pytz 패키지의 도움을 받아 훌륭하게 동작.

* datetime이 최선의 선택이고, time을 사용하지 말아야 하는 이유를 완전히 이해하는 것이 이번 장의 내용이다.
<hr>

## time 모듈
### 1. UTC 시간에서 지역시간으로 변환하는 경우
* 내장 모듈 time의 **localtime 함수**는 유닉스 타임스탬프(UTC에서 유닉스 기원, epoch 이후 지난 초)를 호스트 컴퓨터의 시간대(코랩 시간대는 그냥 UTC 인가부다)와 일치하는 지역 시간으로 변환

In [0]:
import logging

In [0]:
# localtime: UTC 타임스탬프 => 지역시간

from time import localtime, strftime, time

# now = 1407694710
now = time()
print('now is', now)

local_tuple = localtime(now)
time_format = '%Y-%m-%d %H:%M:%S'
time_str = strftime(time_format, local_tuple)
print(time_str)

now is 1570892479.4142067
2019-10-12 15:01:19


In [0]:
! date # 한국 시간대로 설정이 되어 있으면, UTC 대신 KST(KOREA STANDARD TIME) 라고 나옴

Sat Oct 12 15:01:42 UTC 2019


In [0]:
! more /etc/timezone

Etc/UTC


In [0]:
!tzselect

Please identify a location so that time zone rules can be set correctly.
Please select a continent, ocean, "coord", or "TZ".
 1) Africa
 2) Americas
 3) Antarctica
 4) Asia
 5) Atlantic Ocean
 6) Australia
 7) Europe
 8) Indian Ocean
 9) Pacific Ocean
10) coord - I want to use geographical coordinates.
11) TZ - I want to specify the time zone using the Posix TZ format.
#? 4
Please select a country whose clocks agree with yours.
 1) Afghanistan		  18) Israel		    35) Palestine
 2) Armenia		  19) Japan		    36) Philippines
 3) Azerbaijan		  20) Jordan		    37) Qatar
 4) Bahrain		  21) Kazakhstan	    38) Russia
 5) Bangladesh		  22) Korea (North)	    39) Saudi Arabia
 6) Bhutan		  23) Korea (South)	    40) Singapore
 7) Brunei		  24) Kuwait		    41) Sri Lanka
 8) Cambodia		  25) Kyrgyzstan	    42) Syria
 9) China		  26) Laos		    43) Taiwan
10) Cyprus		  27) Lebanon		    44) Tajikistan
11) East Timor		  28) Macau		    45) Thailand
12) Georgia		  29) Malaysia		    46) Turkmenistan
13) Hong

In [0]:
! export TZ='Asia/Seoul'
! echo $TZ
! date


Sat Oct 12 15:22:39 UTC 2019


In [0]:
! echo $env

### 2. 지역 시간으로 사용자 입력을 받아서 UTC 시간으로 변환 하는 경우
* **strptime 함수**로 시간 문자열을 파싱한 후에 **mktime 함수**로 지역 시간을 유닉스 타임스탬프로 변환.

In [0]:
from time import mktime, strptime

print('time_str', time_str)
print('time_format', time_format)
time_tuple = strptime(time_str, time_format)
utc_now = mktime(time_tuple)
print('utc_now', utc_now)

time_str 2019-10-12 15:01:19
time_format %Y-%m-%d %H:%M:%S
utc_now 1570892479.0


In [0]:
time_tuple

time.struct_time(tm_year=2019, tm_mon=10, tm_mday=12, tm_hour=15, tm_min=1, tm_sec=19, tm_wday=5, tm_yday=285, tm_isdst=-1)

### 3. PDT로 파싱하는 코드(=> time 모듈을 사용하면 안되는 케이스)
* PST - 태평양 연안 표준시
* PDT - 태평양 연안 표준시 - 썸머타임
* EST - 동부 표준시
* EDT - 동부 표준시 - 썸머타임

In [0]:
parse_format = '%Y-%m-%d %H:%M:%S %Z' # %Z가 매칭이 안됨...
depart_sfo = '2014-05-01 15:45:16 PDT'
time_tuple = strptime(depart_sfo, parse_format)
time_str = strftime(time_format, time_tuple)
print(time_str)

ValueError: ignored

In [0]:
parse_format = '%Y-%m-%d %H:%M:%S'
depart_sfo = '2014-05-01 15:45:16'
time_tuple = strptime(depart_sfo, parse_format)
time_str = strftime(time_format, time_tuple)
print(time_str)

2014-05-01 15:45:16


### 4. EDT로 파싱하는 코드(=> time 모듈을 사용하면 안되는 케이스)

In [0]:
try:
    arrival_nyc = '2014-05-01 23:33:24 EDT'
    time_tuple = strptime(arrival_nyc, time_format)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-16-2a32494428f3>", line 3, in <module>
    time_tuple = strptime(arrival_nyc, time_format)
  File "/usr/lib/python3.6/_strptime.py", line 559, in _strptime_time
    tt = _strptime(data_string, format)[0]
  File "/usr/lib/python3.6/_strptime.py", line 365, in _strptime
    data_string[found.end():])
ValueError: unconverted data remains:  EDT


* 문제는 플랫폼에 의존적인 time 모듈의 특성이다. 
* 실제 동작은 내부의 C 함수가 호스트 운영체제와 어떻게 동작하느냐에 따라 결정된다. 
* 이와 같은 동작 때문에 파이썬의 time 모듈의 기능을 신뢰하기 어렵다.

* 따라서 이런 목적으로는 time 모듈을 사용하지 말아야 한다. 
* time을 사용해야 한다면 UTC와 호스트 컴퓨터의 지역 시간을 변환하는 목적으로만 사용해야 한다. 
* 다른 형태의 변환에는 datetime 모듈을 사용해야 한다.

##  datetime과 pytz 모듈
* 파이썬에서 시간을 표현하는 두 번째 방법은 내장 모듈 datetime의 datetime 클래스를 사용하는 것. 
* time 모듈과 마찬가지로 datetime은 UTC에서의 현재 시각을 지역 시간으로 변경하는 데 사용할 수 있다.

### 1. 현재 시각을 UTC로 얻어와서 호스트의 지역시간(코랩에서는 변화 없음...)으로 변경하는 경우

In [0]:
from datetime import datetime, timezone

now = datetime(2014, 8, 10, 18, 18, 30)
# now = datetime.now()

now_utc = now.replace(tzinfo=timezone.utc)
now_local = now_utc.astimezone()
print(now_local)

2014-08-10 18:18:30+00:00


### 2. 지역 시간을 UTC로 변경하는 경우

In [0]:
time_str = '2014-08-10 11:18:30'
now = datetime.strptime(time_str, time_format)
time_tuple = now.timetuple()
utc_now = mktime(time_tuple)
print(utc_now)

1407669510.0


In [0]:
dir(timezone)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getinitargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'dst',
 'fromutc',
 'max',
 'min',
 'tzname',
 'utc',
 'utcoffset']

* datetime 모듈은 time 모듈과 달리 한 지역 시간을 다른 지역 시간으로 신뢰성 있게 변경
* 하지만 tzinfo 클래스와 관련 메소드를 이용한 시간대 변환 기능만 제공한다. 
* 빠진 부분은 UTC 이외의 시간대 정의다(dir(timezone) 찍어보면 utc 밖에 없음).

* 다행히도 파이썬 커뮤니티에서는 이 허점을 pypi에서 다운로드할 수 있는 pytz 모듈로 해결하고 있다. 
* pytz는 필요한 모든 시간대에 대한 정의를 담은 전체 데이터베이스를 포함한다.

* pytz를 효과적으로 사용하려면 항상 지역 시간을 UTC로 먼저 변경해야 한다. 
* 그러고 나서 UTC 값에 필요한 datetime 연산(오프셋 지정 등)을 수행한다. 
* 그런 다음 마지막 단계로 지역 시간으로 변환한다.
### 3. pytz 모듈을 사용하여 서로다른 시간대 사이 변환하기
* A시간 => UTC 시간 => B시간 이런 식으로 변환해야 함.

In [0]:
# 미국 동부 시간 => UTC
import pytz

arrival_nyc = '2014-05-01 23:33:24'  # 뉴욕 도착 시간
nyc_dt_naive = datetime.strptime(arrival_nyc, time_format)
eastern = pytz.timezone('US/Eastern')
nyc_dt = eastern.localize(nyc_dt_naive)
utc_dt = pytz.utc.normalize(nyc_dt.astimezone(pytz.utc))
print(utc_dt)

2014-05-02 03:33:24+00:00


In [0]:
# UTC => 미국 서부 시간
pacific = pytz.timezone('US/Pacific')
sf_dt = pacific.normalize(utc_dt.astimezone(pacific))
print(sf_dt)

2014-05-01 20:33:24-07:00


In [0]:
# UTC => 네팔 카트만두
nepal = pytz.timezone('Asia/Katmandu')
nepal_dt = nepal.normalize(utc_dt.astimezone(nepal))
print(nepal_dt)

2014-05-02 09:18:24+05:45


In [0]:
# UTC => 한국 서울 시간
seoul = pytz.timezone('Asia/Seoul')
seoul_dt = seoul.normalize(utc_dt.astimezone(seoul))
print(seoul_dt)

2014-05-02 12:33:24+09:00


# 핵심 정리
* 서로 다른 시간대를 변환하는 데는 time 모듈을 사용하지 말자.
* pytz 모듈과 내장 모듈 datetime으로 서로 다른 시간대 사이에서 시간을 신뢰성 있게 변환하자.
* 항상 UTC로 시간을 표현하고, 시간을 표시하기 전에 마지막 단계로 UTC 시간을 지역 시간으로 변환하자